In [ ]:
import pandas as pd
from kaos.visualization import set_chart
from kaos.data.loading import CSVPreset
from kaos.data.data import Data, ReferenceData
from pathlib import Path

from dotenv import load_dotenv
import os
from kaos.data.instruments import merge_contracts, FuturesContract
from kaos.data.enums import AssetClass, DataProvider
import hvplot.pandas

load_dotenv()

True

## lettura di 6EH2024 daily da firstrate

In [ ]:
DATA_PATH = Path(os.getenv("DATA_PATH"))
product = "E6"
month_code = "H"
year = 2024
multiplier = "1"
resolution = "day"
extension = ".txt"
symbol = product + month_code + str(year)
filename = f"{product}_{month_code}{year % 2000}_{multiplier}{resolution}{extension}"
daily_dir = "market/ohlc/csv/firstrate/indi_arch_fut_1d"
# TODO use the same timeframe naming convention for both directory and files
some_contract: Path = Path(DATA_PATH / daily_dir / filename)

In [ ]:
df: pd.DataFrame = Data.get_csv(some_contract, CSVPreset.PRESET_FIRSTRATE)
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 1271 entries, 2019-03-05 00:00:00-05:00 to 2024-03-18 00:00:00-04:00
Data columns (total 6 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   open           1271 non-null   float64
 1   high           1271 non-null   float64
 2   low            1271 non-null   float64
 3   close          1271 non-null   float64
 4   volume         1271 non-null   int64  
 5   open_interest  1271 non-null   int64  
dtypes: float64(4), int64(2)
memory usage: 69.5 KB


In [ ]:
euro_contracts = []
for year in range(2019, 2024):
    for month_code in "HMUZ":
        filename = (
            f"{product}_{month_code}{year % 2000}_{multiplier}{resolution}{extension}"
        )
        path = Path(DATA_PATH / daily_dir / filename)
        df = Data.get_csv(path, CSVPreset.PRESET_FIRSTRATE)
        symbol = product + month_code + str(year)
        ref = ReferenceData(provider=DataProvider.FIRSTRATE, asset_class=AssetClass.FX)
        contract = FuturesContract(
            reference_data=ref, symbol=symbol, market_data={"D": df}
        )
        euro_contracts.append(contract)

euro_contracts

[E6H2019,
 E6M2019,
 E6U2019,
 E6Z2019,
 E6H2020,
 E6M2020,
 E6U2020,
 E6Z2020,
 E6H2021,
 E6M2021,
 E6U2021,
 E6Z2021,
 E6H2022,
 E6M2022,
 E6U2022,
 E6Z2022,
 E6H2023,
 E6M2023,
 E6U2023,
 E6Z2023]

## genera il continuous contract

In [36]:
from kaos.data.enums import RolloverRule


continuous = merge_contracts(euro_contracts, RolloverRule.EXPIRY)
# continuous.market_data["D"].info()
continuous.market_data

,open,high,low,close,volume,open_interest
timestamp,,,,,,
2017-12-18 00:00:00-05:00,1.22700,1.22700,1.22605,1.22605,4,26
2017-12-19 00:00:00-05:00,1.22800,1.23195,1.22800,1.23195,1,27
2017-12-20 00:00:00-05:00,1.23200,1.23485,1.23200,1.23485,4,31
2017-12-21 00:00:00-05:00,1.23360,1.23460,1.23360,1.23460,10,41
2018-01-02 00:00:00-05:00,1.25000,1.25120,1.25000,1.25120,20,49
...,...,...,...,...,...,...
2023-12-12 00:00:00-05:00,1.07680,1.08325,1.07635,1.07925,385335,476930
2023-12-13 00:00:00-05:00,1.07970,1.08990,1.07750,1.08900,542261,252243
2023-12-14 00:00:00-05:00,1.08765,1.10105,1.08765,1.09930,388462,187676


grafico plotly generato da chata

In [37]:
from holoviews.plotting.links import RangeToolLink

df = continuous.market_data
ohlc = df.hvplot.ohlc(grid=False, height=300)
oi = df.hvplot.line(y="open_interest", height=150)
# RangeToolLink(ohlc.get(0), oi.get(0))
layout = (ohlc + oi).cols(1).opts(shared_axes=True)
layout

:Layout
   .Overlay.I :Overlay
      .Segments.I   :Segments   [timestamp,low,timestamp,high]   (open,high,low,close,lbound,ubound)
      .Rectangles.I :Rectangles   [lbound,open,ubound,close]   (open,high,low,close,lbound,ubound)
   .Curve.I   :Curve   [timestamp]   (open_interest)

In [38]:
from kaos.analysis.pdas import Fvg


continuous.data["bisi"], continuous.data["sibi"] = Fvg.formed(continuous.data)
continuous.data

AttributeError: 'FuturesContract' object has no attribute 'data'

In [ ]:
chart = set_chart(continuous.data)
chart.load()